In [18]:
import geopandas as gpd
import folium

# Read the GeoJSON file
geojson_file = 'geojson.json'
gdf = gpd.read_file(geojson_file)

# Create a map centered around the centroid of the GeoDataFrame with Stamen Watercolor style
center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
m = folium.Map(location=center, zoom_start=14)

# Add the GeoDataFrame to the map
folium.GeoJson(gdf).add_to(m)

# Display the map
m

/var/folders/zp/r_synnss3dnc2lgwfbyr07h80000gn/T/ipykernel_33928/862522175.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
